In [ ]:
# Parameters
day_obs = '2024-06-26'
instrument = 'latiss'

<a class="anchor" id="imports"></a>
## Imports and General Setup

In [ ]:
import os
from lsst.summit.utils import ConsDbClient
from lsst.summit.utils.utils import computeCcdExposureId
import sqlalchemy
import requests
import pandas as pd

URL = "https://usdf-rsp.slac.stanford.edu/consdb/" # Need to add that part about the headers to client flow through
URL = "http://consdb-pq.consdb:8080/consdb" # Don't use this one

os.environ["no_proxy"] += ",.consdb"

access_token = os.getenv("ACCESS_TOKEN")
headers = {"Authorization": f"Bearer {access_token}"}

sesh = requests.Session()
sesh.headers.update(headers)

%matplotlib inline

In [ ]:
try:
    from lsst.summit.utils import ConsDbClient
    have_consdb = True
except ImportError:
    have_consdb = False

if have_consdb:
    client = ConsDbClient(URL)

    print(client.schema())  # list the instruments
    print(client.schema('latiss'))  # list tables for an instrument
    print(client.schema('latiss', 'cdb_latiss.exposure_flexdata')) # specifically flexdata table

<a class="anchor" id="get_records"></a>
## Get Records

In [ ]:
# Add consdb
day_obs_int = int(day_obs.replace('-', ''))

visit_query1 = f'''
    SELECT * FROM cdb_{instrument}.visit1
     where day_obs = {day_obs_int}
'''

instrument = 'lsstcomcam'
visit_query2 = f'''
    SELECT * FROM cdb_{instrument}.visit1
     where day_obs = {day_obs_int}
'''

instrument = 'lsstcomcamsim'
visit_query3 = f'''
    SELECT * FROM cdb_{instrument}.visit1
     where day_obs = {day_obs_int}
'''

#what is a quicklook
quicklook_query = f'''
    SELECT q.*  FROM cdb_{instrument}.visit1_quicklook as q,
    cdb_{instrument}.visit1 as v
     WHERE v.day_obs = {day_obs_int} and q.visit_id = v.visit_id
'''

if have_consdb:
    # Use the ConsDB Client, and add a couple of tries
    consdb = ConsDbClient(URL)

    try:
        visits_latiss = consdb.query(visit_query1).to_pandas()
        visits_lsstcc = consdb.query(visit_query2).to_pandas()
        visits_lsstccs = consdb.query(visit_query3).to_pandas()

    except requests.HTTPError or requests.JSONDecodeError:
        # Try twice
        visits_latiss = consdb.query(visit_query1).to_pandas()
        visits_lsstcc = consdb.query(visit_query2).to_pandas()
        visits_lsstccs = consdb.query(visit_query3).to_pandas()

    quicklook = consdb.query(quicklook_query).to_pandas()

else:
    # Assumes at the USDF
    connection = sqlalchemy.create_engine('postgresql://usdf@usdf-summitdb.slac.stanford.edu/exposurelog')
    visits_latiss = pd.read_sql(visit_query1, connection)
    quicklook = pd.read_sql(quicklook_query, connection)

if len(visits_latiss) > 0:
    print(f"Retrieved {len(visits_latiss)} visits from consdb")
    obj_vis = len(visits_latiss.query('img_type == "OBJECT"'))
    print(f"About {obj_vis} of these are object images")

if len(quicklook) > 0:
    visits_latiss = visits_latiss.join(quicklook, on='visit_id', lsuffix='', rsuffix='_q')
    print(f"And added quicklook stats")


## The schemas in cdb are
`latiss`, `lsstcomcam`, `lsstcomcamsim`

## And each have the following tables
`exposure`, `visit1`

`ccdexposure`, `ccdvisit1`

`ccdexposure_camera`

`visit1_quicklook`, `ccdvisit1_quicklook`

 `exposure_flexdata`, `ccdexposure_flexdata`

 ## We can find more info on the schemas at https://sdm-schemas.lsst.io/

<a class="anchor" id="table"></a>
## Tables of (mostly raw) results

In [ ]:
data = pd.DataFrame(visits_latiss)
print(f"Latiss has {len(data.columns)} columns")
print(f"They are {data.columns}")

In [ ]:
data

In [ ]:
pd.DataFrame(visits_lsstcc)
#Yes this is empty

In [ ]:
pd.DataFrame(visits_lsstccs)

# Rapid analysis was mentioned
ccdvisit1_quicklook holds some psf* attributes
quicklook might mean rapid analysis

In [ ]:
print(f"Many Quicklook columns: {len(quicklook.columns)}")
print(f"They include: {quicklook.columns}")

In [ ]:
quicklook